## 0.Package Import

In [8]:
import os #For managing the operating system
import requests #For getting files
import math

import numpy as np #For arrays #Math and matrices
import pandas as pd #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from IPython.core.display import HTML

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import collections
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import gensim
from gensim import corpora, models
from gensim.models import Word2Vec
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

import spacy
import en_core_web_sm
import seaborn as sns

#% matplotlib inline
pd.options.mode.chained_assignment = None

from tqdm import tqdm
from tqdm.auto import tqdm
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import openai

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [9]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...


True

In [ ]:
#openai.api_key = os.environ["OPENAI_API_KEY"]
# openai.api_key = ''
OPENAI_API_KEY = ''


## 1.LDA

## 1.1 Find Best LDA model

In [ ]:
# lda parameter tuning

# Topics range
min_topics = 5
max_topics = 20 #12

rnd_seed = 42
np.random.seed(rnd_seed) ## P0066 and P0007 didn't use this seed. 

model_results = {'Topics': [],
                 'Coherence': []
                #  'perplexity': []
                 }

# # Can take a long time to run
# pbar = tqdm(total=10)

# iterate through validation corpuses
for i in range(min_topics, max_topics):
    # get the coherence score for the given parameters ##TODO: don't use function, build lda model, calculate perplexity and cv following that. Record perplexity as well. 
      
        lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word= id2word,
                                           num_topics= i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        coherence_model_lda = CoherenceModel(
            model=lda_model, texts=texts, dictionary=id2word, coherence='u_mass')  
        cv = coherence_model_lda.get_coherence()

        # perplexity = lda_model.log_perplexity(bow_corpus)
                                
        # Save the model results
        model_results['Topics'].append(i)
        model_results['Coherence'].append(cv)
        # model_results['perplexity'].append(perplexity)
        
        # pbar.update(1)
# pbar.close()

In [ ]:
## Save results
res_dir = 'C:/dev/'#'D:/dev/Python/data/results/'
res_name = 'lda_tuning_results_' + PSO[k] + '.csv'
pd.DataFrame(model_results).to_csv(os.path.join(res_dir, res_name), index=False)

### CV Score

In [ ]:
# plot
# pick the model that gave the highest CV

plt.figure()
# for valid in valid_set:
#     selected_data = results_for_vis.loc[results_for_vis['Validation_Set'] == valid]
#     plt.plot(selected_data['Topics'], selected_data['Coherence'], label=valid)

plt.plot(model_results['Topics'], model_results['Coherence'], label = 'Coherence')
# plt.plot(model_results['Topics'], model_results['perplexity'], label = 'perplexity')

# plt.ylabel('Coherence Score')
plt.xlabel('Number of Topics')
plt.legend()

fig_coh_name =  PSO[k] + '_coh.png'
print(fig_coh_name)
plt.savefig(res_dir + '/' + fig_coh_name)
# plt.savefig('P0066_coh.png')

plt.show()



### LDA model results

In [ ]:
## Test number of topics
## Test the one with the lowest u_mass coherence score for each PSO

num_topics = 11


In [ ]:
## Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word= id2word,
                                           num_topics= num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.show_topics(num_topics=-1) ## show all topics


## 1.2 Document topic and percent contribution for each document

In [ ]:
# check what is the Dominant topic and its percentage contribution in each document
def format_topics_sentences(corpus, cleaned_texts, ldamodel=None):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series(
                    [int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic',
                              'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(cleaned_texts)
    # orig_texts = pd.Series(orig_texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    # sent_topics_df = pd.concat([sent_topics_df, orig_texts], axis=1)

    ## Generate output dataframe
    text_orig = text_original.reset_index()['ANSWER_TEXT']
    df_topic_sents_keywords = pd.concat([sent_topics_df, text_orig], axis = 1)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = [
        'Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'ProcText', 'OrigText']
    df_dominant_topic.head(10)
    
    ## save results to output
    filename = 'topic_sentences'+ PSO[k] + '.csv'
    pd.DataFrame(df_dominant_topic).to_csv(os.path.join(res_dir, filename), index=False)

    return(df_topic_sents_keywords)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(corpus=bow_corpus, cleaned_texts=texts, ldamodel=lda_model)

## 1.3 Representative sentence for each topic

In [ ]:
# check the most representative sentence for each topic
# Display setting to show more characters in column

def get_represent_topic_sent(df_topic_sents_keywords):

    pd.options.display.max_colwidth = 100

    sent_topics_sorteddf_mallet = pd.DataFrame()
    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                                grp.sort_values(['Perc_Contribution'], ascending=False).head(1)],
                                                axis=0)

    # Reset Index
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

    # Format
    sent_topics_sorteddf_mallet.columns = [
        'Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text", "orig_texts"]

    # Show
    sent_topics_sorteddf_mallet

    ## save results to output
    filename = 'topic_sentence_representatives' + PSO[k] + '.csv'
    pd.DataFrame(sent_topics_sorteddf_mallet).to_csv(os.path.join(res_dir, filename), index=False)


In [ ]:
get_represent_topic_sent(df_topic_sents_keywords)

### Word counts and importance of topic words

In [ ]:
def plot_topic_word_importance(lda_model, dat_tok_cleaned_test, num_topics):

    topics = lda_model.show_topics(formatted=False, num_topics=-1)
    data_flat = [w for w_list in dat_tok_cleaned_test for w in w_list]
    counter = Counter(data_flat)

    out = []
    for i, topic in topics:
        for word, weight in topic:
            out.append([word, i, weight, counter[word]])

    df = pd.DataFrame(
        out, columns=['word', 'topic_id', 'importance', 'word_count'])

    # Plot Word Count and Weights of Topic Keywords
    num_rows = math.ceil(num_topics/3)
    fig, axes = plt.subplots(num_rows, 3, figsize=(16, 10), sharey=True, dpi=160)
    col_repeats = plt.cm.get_cmap('tab20').colors
    cols = col_repeats #[color for name, color in mcolors.TABLEAU_COLORS.items()]
    for i, ax in enumerate(axes.flatten()):
        if(i < num_topics):
            ax.bar(x='word', height="word_count",
                data=df.loc[df.topic_id == i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
            ax_twin = ax.twinx()
            ax_twin.bar(x='word', height="importance",
                        data=df.loc[df.topic_id == i, :], color=cols[i], width=0.2, label='Weights')
            ax.set_ylabel('Word Count', color=cols[i])
            ax_twin.set_ylim(0, 0.030)
            ax.set_ylim(0, 10000)
            ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
            ax.tick_params(axis='y', left=False)
            ax.set_xticklabels(df.loc[df.topic_id == i, 'word'],
                            rotation=30, horizontalalignment='right')
            ax.legend(loc='upper left')
            ax_twin.legend(loc='upper right')

    fig.tight_layout(w_pad=2)
    fig.suptitle('Word Count and Importance of Topic Keywords',
                fontsize=22, y=1.05)

    fig_coh_name =  'Topic_word_count_importance' + PSO[k] + '.png'
    print(fig_coh_name)
    plt.savefig(res_dir + '/' + fig_coh_name)

    plt.show()


In [ ]:
plot_topic_word_importance(lda_model, dat_tok_cleaned_test, num_topics)

In [ ]:
df

## 1.4 T-SNE Clustering

In [ ]:
# Get topic weights and dominant topics ------------
# t-SNE Clustering Chart - visualize the clusters of documents in a 2D space using t-SNE (t-distributed stochastic neighbor embedding) algorithm.
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[bow_corpus]):
    topic_weights.append([w for i, w in row_list])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 6
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

In [ ]:
import pyLDAvis.gensim_models
import pickle
import pyLDAvis

In [ ]:
## Visualize the topics
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dic_lda)

In [ ]:
pyLDAvis.save_html(p, 'lda_test.html')

In [ ]:
p

# 2.LLMs (Run on Google Colab)

## 2.1 OpenAI Classification Task

In [ ]:
# Read the CSV file from Google Drive
df = pd.read_csv('/content/drive/MyDrive/uchicago/NLP/data/final_df_withtoken.csv')

In [ ]:
# First try using sklearn to identify benefits and challenges in the df
from sklearn.feature_extraction.text import CountVectorizer

# Define keywords for benefits and challenges
benefits_keywords = ['access to care', 'efficiency', 'effective','cost-effective',"accessibility", "convenience","cheap"]
challenges_keywords = ['concern', 'difficulties', 'limited',"technical issues", "challenge",'difficult','expensive','lack','risk',"breach"]

# Initialize CountVectorizer to transform texts into binary variables based on keywords
vectorizer = CountVectorizer(vocabulary=benefits_keywords + challenges_keywords, binary=True)
keyword_features = vectorizer.fit_transform(df['text'])

# Convert to DataFrame and concatenate with the original DataFrame
keywords_df = pd.DataFrame(keyword_features.toarray(), columns=vectorizer.get_feature_names_out())
df = pd.concat([df, keywords_df], axis=1)

# Create binary variables for each set of keywords (1 if any keyword is present, 0 otherwise)
df['benefits'] = df[benefits_keywords].sum(axis=1) > 0
df['challenges'] = df[challenges_keywords].sum(axis=1) > 0

# Create a treatment variable based on the presence of benefits vs. challenges keywords
df['treatment_focus'] = np.where(df['benefits'], 1,
                                               np.where(df['challenges'], -1, 0))

In [ ]:
df.treatment_focus.value_counts()

In [ ]:
def truncate_text(text, max_length=4097):
    """Truncate text to ensure it fits within a specified maximum length."""
    return text if len(text) <= max_length else text[:max_length]

def classify_text_with_openai(text):
    truncated_text = truncate_text(text, max_length=4097)  # Adjust max_length as needed
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Is this text discussing the benefits or challenges of telehealth? Return only one word, 'benefit' or 'challenges' or 'neither', Text: '{truncated_text}'"}
        ]
    )
    # Correctly access the response data
    return response.choices[0].message.content.strip()

# Apply the classification function to each text in your sample
df_subset['openai_classification'] = df_subset['text'].apply(classify_text_with_openai)

In [ ]:
df_subset['openai_classification']

In [ ]:
#df_subset.to_csv('/content/drive/MyDrive/uchicago/NLP/data/df_subset.csv', index=False)
df=pd.read_csv('/content/drive/MyDrive/uchicago/NLP/data/df_subset.csv')

In [ ]:
df['openai_classification'] = df['openai_classification'].apply(lambda x: x.lower())
df['openai_classification'] = df['openai_classification'].apply(lambda x: 0 if x == 'neither' else 1 if x == 'benefit' else -1)



In [ ]:
df.openai_classification.value_counts()

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_classification_multiclass(true_labels, predictions, method_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted') #macro

    print(f"Results for {method_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Weighted Avg.): {precision:.4f}")
    print(f"Recall (Weighted Avg.): {recall:.4f}")
    print(f"F1 Score (Weighted Avg.): {f1:.4f}\n")


In [ ]:
evaluate_classification_multiclass(df['true_label'], df['openai_classification'], 'OpenAI API Classification')
evaluate_classification_multiclass(df['true_label'], df['treatment_focus'], 'Transformer Classification')


### Huggingface Login

In [ ]:
! git config --global credential.helper store

In [ ]:
!huggingface-cli login

### Model Config

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

## 2.2 Comparing Fine-tune LLM

In [ ]:
# change base model ?
# shorten text

dataset = load_dataset('csv', data_files='/content/drive/MyDrive/uchicago/NLP/data/df_subset_short.csv',split="train")
# Enable GPU training if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32, # the rank of the adaptation
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True, # speed up
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

# Train model
trainer.train()



# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(base_model).to(device)
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Training parameters with mixed precision (fp16) enabled
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,  # Adjust based on your GPU memory and desired effective batch size
    optim="adamw_torch",  # Consider using the default optimizer for simplicity
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    save_total_limit=1,  # Limit the number of saved model checkpoints
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_params,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

In [ ]:
# Fine-tune the LLM
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/uchicago/NLP/data/df_subset.csv',split="train")
model = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)
trainer.train()
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Evaluate the performance of the OpenAI LLM, the fine-tuned LLM, and the "vanilla" un-tuned LLM
openai_results = evaluate_openai_llm(dataset)
vanilla_results = evaluate_vanilla_llm(dataset)
fine_tuned_results = evaluate_fine_tuned_llm(dataset)

# Compare the performance of the models
print("OpenAI LLM:")
print(openai_results)
print("Vanilla LLM:")
print(vanilla_results)
print("Fine-tuned LLM:")
print(fine_tuned_results)


## 2.3 Synthesize Data with LLMs

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/uchicago/NLP/data/df_subset.csv')
# Define the prompt template for generating telehealth-related discussions
def telehealth_language_nuance_prompt() -> list:
    system_message = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],
            template="You are a model trained on telehealth discussions, generating texts that reflect the nuances in telehealth discourse."
        )
    )
    human_message = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["text", "num_generations"],
            template="""Rewrite the following text {num_generations} times, keeping the focus on telehealth. 
            Aim to highlight aspects such as accessibility, efficiency, and patient satisfaction. 
            Text: {text}
            """
        )
    )
    return [system_message, human_message]

OPENAI_API_KEY = ''  # Initialize the LLM with API key
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9, max_tokens=512, openai_api_key=OPENAI_API_KEY)

# Function to generate augmented texts based on the telehealth discussions
def generate_augmented_texts(df):
    generated_texts = []
    for i, row in tqdm(df.iterrows(), total=len(df)):
        text = row["text"]
        prompt = ChatPromptTemplate.from_messages(telehealth_language_nuance_prompt())
        chain = LLMChain(prompt=prompt, llm=llm)

        augmented_text = chain.run({"text": text, "num_generations": 1})
        generated_texts.append(augmented_text)

    df["augmented_text"] = generated_texts
    return df

# Apply the function to generate augmented texts
df_augmented = generate_augmented_texts(df)

# Example analysis: Comparing the frequency of the word "efficiency" in model-generated vs. actual data
efficiency_synthetic = len([k for k in df_augmented["augmented_text"] if "efficiency" in k.lower()]) / len(df_augmented["augmented_text"])
efficiency_real = len([k for k in df["text"] if "efficiency" in k.lower()]) / len(df["text"])

print(f"'Efficiency' present in {round(efficiency_synthetic, 3) * 100}% of synthetic texts")
print(f"'Efficiency' present in {round(efficiency_real, 3) * 100}% of real texts")


## 2.4 Few-Shots Prediction

In [ ]:
from langchain.llms import OpenAI

# Initialize the OpenAI model
model = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=OPENAI_API_KEY)

# Prepare the prompt with the DataFrame information included
prediction_prompt = """
Given some data on news about telehealth, examples are as follows:
1.One patient succumbs to brain stroke every four minutes in India says expert Brain stroke is the second most common cause of death in India with one patient succumbing to the disease every four minutes , a top health expert flagged on Thursday . Advertisement Padma Shri awardee Dr ( Prof ) MV Padma Srivastava , who is the most renowned neurologist in the country and is a Professor of Neurology at All India Institute of Medical Sciences , participated in the celebration of International Women 's Day event at Sir Ganga Ram Hospital today . Delivering a keynote address at the event titled , " Stroke care and its primary preventive methods in poor resource settings in India " , Dr Srivastava said , " Stroke is the second most common cause of death in India . About 1,85,000 strokes occur every year in India with nearly one stroke every 40 seconds and one stroke death every 4 minutes . " She further referred to the Global Burden of Diseases ( GBD ) and said that most incidents of stroke were recorded in burden of stroke with 68.6 per cent incidence of stroke . 70.9 per cent stroke deaths and 77.7 Disability Adjusted Life Years ( DALYs ) lost . These figures are alarming for India with many living in poor resource settings . Another alarming and important finding of the GBD 2010 stroke project is 5.2 million ( 31 per cent ) strokes were in children aged less than 20 years . The stroke burden is greater in India and more so among younger and middleaged people , " she said . Advertisement The health expert flagged the lack of necessary infrastructure to deal with the alarming data in the country . " In spite of these alarming figures , many Indian hospitals lack the necessary infrastructure and organization required to treat stroke patients quickly and efficiently and do not deliver adequate stroke care . The stroke services across the country especially in public sector hospitals are deficient in many aspects , " Dr Srivastava said . The Padma Shri awardee further listed the solutions to boost the infrastructure needed to tackle the rising numbers . " poor resource settings in India is to adopt Telestroke models in poor resource settings . Implementation of Telemedicine / Telestroke facilities is an important step for bridging the economically and geographically challenged and underprivileged sections of the society , " she said . This program also included inspirational talks by three distinguished faculty members of the hospital . Advertisement The members included Dr Jayashree Sood , Chairperson , Institute of Anesthesiology , who spoke on how to maintain the balance between work and life , especially for women , Prof Kusum Verma , Advisor Cytopathology who spoke on her experiences mitigating professional challenges and Padma Bhushan Dr Neelam Kler Chairperson of Department of Neonatology who talked about her belief in the saying ' Never say Never ' . This program was hosted by the department of Research and chaired by Prof NK Ganguly , former director general ICMR and Chairperson department of Research , Sir Ganga Ram Hospital . The department of research at Sir Ganga Ram Hospital has a robust PhD program and a state of art equipped laboratory with a focus on basic biology , stem cell biology , immunology , autoimmune and infectious diseases .
2.The Wearable Technologies Improving Life for People With Parkinson 's Disease Wearable health technologies are vastly popular with people wanting to improve their physical and mental health . Everything from exercise , sleep patterns , calories consumed and heart rhythms can be tracked by a wearable device . But timely and accurate data is also especially valuable for doctors treating patients with complicated health conditions using virtual care . A new study from the Southern Medical Program ( SMP ) , based at UBC Okanagan , has examined the use of wearable health technology and telehealth to treat patients with Parkinson 's disease . " Even prior to the pandemic , telehealth helped deliver specialized care to patients living in remote and rural settings , " says Wile , a clinical investigator with the Centre for Chronic Disease Prevention and Management . " But with the complex nature of Parkinson 's , we wanted to enhance these appointments to better understand how movements vary throughout a patient 's entire day . " To add a new layer of health information , Wile and the " We recruited Parkinson 's patients with either tremors or involuntary movements , " says Joshua Yoneda , SMP student and coauthor of the study . " We then divided them into two groups some using telehealth and devicebased health tracking and others attending traditional facetoface appointments . " The telehealth group wore wearable devices to track their movements , involuntary or not , throughout waking hours . The reported data was then reviewed during telehealth appointments to identify peak times patients experienced Parkinson 's symptoms . " With the integration of accurate and reliable data from wearable devices , we were able to tailor a patient 's medication to better manage their symptoms throughout the day , " adds Wile . As part of the study , patients were asked a series of questions from the standardized Parkinson Disease Quality of Life Index . Both study groups were assessed at intervals of six weeks , three months and six months . Overall , the patients using the wearable devices reported positive experiences and health outcomes in combination with telehealth appointments to access specialized care leverage multiple technologies to improve a patient 's quality of life and limit the added stress and cost associated with travel , " says Yoneda . Reference : Peacock D , Yoneda J , Thomson V , Wile D. Tailoring the use of wearable systems and telehealth for Parkinson 's disease . Parkinsonism & Related Disorders . 2021 ; 89:111112. **40;409;TOOLONG article has been republished from the following materials . Note : material may have been edited for length and content . For further information , please contact the cited source .
3.Building smarter healthcare in HK with intelligent automation That Hong Kong 's healthcare system had been under enormous strain is no secret . By the end of March , the territory 's COVID19 numbers had peaked at more than 55,000 infections . Set against the city 's ratio of two doctors per 1,000 patients , far below the ratio in Singapore and Japan ( both at 2.5 ) , the UK ( 3.0 ) and Australia ( 3.8 ) , it becomes obvious that , all things remaining constantan aging population , continued shortage of doctors and nurses and rising costsHong Kong 's buckling healthcare system will need to deliver healthcare services in much smarter and more efficient ways . These trends are reflected in the Hong Kong Hospital Authority 's ( HA ) Strategic Plan 20222027 , which lays out its vision for improving financial stability to " meet escalating service needs " through the development of smart care and smart hospitals , the nurturing of a smart workforce , and enhancing service supply . As implied in the strategic plan , digital solutions underlying by intelligent automation ( IA ) and robotics to further develop and improve services already in play , like telemedicine and telecare , but made smarter and efficient for the long haul . How IA can be an enabler Intelligent automation has been utilised by hospitals to resolve their common problems for a long time . IA enables hospitals and clinics to bring in digital workers AIfuelled software designed to model human roles to execute rulesbased tasks such as appointment bookings and referrals , which go a long way in improving operational excellence and the patient experience . The National University Health System ( NUHS ) in Singapore is a great example of how IA is used to automate processes and improve overall operations . In 2018 , NUHS started to automate their backoffice functions , including claims processing and billing . NUHS has to complete approximately 40,000 bill adjustment requests each year , and through IA , they went from spending an average of three to four days turnaround for task completion to immediate turnaround . The use of IA helped NUHS process 75 cashflow contributing to an estimated US$350,000 savings across three years . Patients seeking reimbursement from their insurers and employers also enjoyed better experience . At the height of the pandemic in April 2020 , Singapore health authorities had to set up testing efforts , particularly at foreign worker dormitories , where more than 1,000 COVID19 tests were administered each day . It was a pressing period when the pandemic was spreading fast , and NUHS was under great pressure to register , test , and share the results quickly across the network . NUHS needed to speed up this process . NUHS leveraged IA in the form of digital workers to improve the efficiency of this laborious process . The results were immediate as test registration time was reduced from two minutes to 30 seconds per test , saving NUHS 18 hours each day . Lab results also arrived more quickly , enabling NUHS to process more than 27,000 patients daily . In addition , as a measure to contain the spread of COVID19 and relieve the load on the stretched healthcare workforce , NUHS leveraged on IA more selfserved and empowered to take charge of their own health through smart portal and remote consultation . Heralding a new era of healthcare The global pandemic has opened avenues and accelerated new ways of working and operating in the healthcare sector . In a Blue Prism survey of more than 400 senior level healthcare professionals across the globe , 93 percent said that automation of processes accelerated because of COVID19 , with 58 percent of respondents replacing paper documents with electronic equivalents , and 57 percent taking the opportunity to build new , automated processes that improved the way they interact with patients and other departments . Almost half ( 45 percent ) said they have replaced inperson consultations with video conferencing , a practice that is likely to continue in the years ahead . In the UK , the vast majority of facetoface consultations that were previously carried out daily shifted to remote consultations . The Royal College of General Practitioners revealed data corroborating this trend , with 71 percent of routine consultations being conducted remotely in the four week leading to mid April in prior , 71 percent of patient meetings were facetoface consultations . Looking ahead , IA will play an increasingly greater role , with machine learning algorithms used to assess patient symptoms , as well as their behaviour , language and expressions . Collaboration technologies will also evolve so patient engagements are more natural and clinicians can pick up on nuances in interactions . Clearly , the future is bright for the adoption of IA and digital solutions , with the healthcare sector a fertile ground for further deployments to drive greater sustainable healthcare for all . For its part , the Hong Kong Government has put special emphasis on healthcare technology with the setting up of InnoHK , a major initiative that houses 16 laboratories for global research collaboration , which also includes innovation in areas like artificial intelligence and robotics technologies . More than ever , healthcare organizations are challenged to do more for patients with the same resources or less . By leveraging IA to power laborious and mundane processes while improving existing solutions like telehealth , it is possible for Hong Kong 's healthcare issues around service availability so that doctors and nurses can finally be freed up to focus and care for patients with urgent medical needs .
4.Access to the abortion pill mifepristone must be restricted , a U.S. appeals court ruled on Wednesday , ordering a ban on telemedicine prescriptions and shipments of the drug by mail , though the decision will not immediately take effect . A Georgia state law against racketeering could be a powerful tool in prosecuting Donald Trump , but applying charges traditionally used to take down organized crime risks miring the case in legal and logistical complications . A campaign aide to embattled Republican U.S. Representative George Santos was charged with identity theft and wire fraud in federal court for impersonating a top congressional staffer in fundraising appeals , court documents unsealed on Wednesday showed . Stay Informed Information you can trust Reuters , the news and media division of Thomson Reuters , is the world 's largest multimedia news provider , reaching billions of people worldwide every day . Reuters provides business , financial , national and international news to professionals via desktop terminals , the world 's media organizations , industry events and directly to consumers .

here are two predictions:

1. Prediction 1 forecasts an increasing emphasis on patient privacy and data security in telehealth.
2. Prediction 2 forecasts the continued growth and enhancement of telehealth offerings in mental health care and chronic disease management.

[Your Task]
Considering these given example news, validate if my the two predictions are correct, and make another prediction about the future of telehealth based on the data provided.
"""

# Use the model to generate a prediction
response = model.generate([few_shot_prompt + prediction_prompt])
print(response)


## 2.5 Sentiment Over Years using LLM

In [ ]:
from openai import OpenAI
import numpy as np
import pandas as pd

class TelehealthResearchLLM:
    def __init__(self, api_key, data):
        self.api_key = api_key
        self.data = data
        openai.api_key = self.api_key
    
    def generate_group_specific_insights(self, country, year):
        """Generate insights on telehealth services' impact and online discourse for a specific country and year."""
        country_data = self.data[(self.data['country'].str.lower() == country.lower()) & (self.data['year'] == year)]
        texts = country_data['text'].tolist()
        prompt = f"Analyze the following telehealth related discussions from {country} in {year} for linguistic representation, sentiment towards telehealth services, and the impact of telehealth on diagnosis and treatment outcomes. Summarize the key findings.\n\nDiscussions:\n\n" + "\n\n".join(texts[:5])  # Limiting to first 5 for brevity
        
        try:
            client = OpenAI(api_key=OPENAI_API_KEY)
            response = client.completions.create(
                model="gpt-3.5-turbo-instruct",  # Use "gpt-4" if available
                prompt=prompt,
                temperature=0.7,
                max_tokens=500
            )
            return response.choices[0].text.strip()
        except Exception as e:
            return str(e)
    
    def analyze_and_compare(self, countries, year):
        """Compare telehealth discourse across different countries for a specific year."""
        insights = {}
        for country in countries:
            insights[country] = self.generate_group_specific_insights(country, year)
        return insights


In [ ]:

#data = pd.read_csv('/content/drive/MyDrive/uchicago/NLP/data/df_subset.csv') 

api_key = ''
research_llm = TelehealthResearchLLM(api_key, data)

# Specify countries and year for comparison
countries = ["United States", "India", "Kenya", "Brazil", "Germany"]
year = 2020 

# Generate and compare insights
insights = research_llm.analyze_and_compare(countries, year)
print("Telehealth Insights:")
for country, insight in insights.items():
    print(f"\nCountry: {country}\nInsight: {insight}")